In [2]:
from IPython.display import display, HTML
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as pyfunc
display(HTML('<style>pre { white-space: pre !important; }</style>'))

In [3]:
spark = SparkSession.builder.appName("answers").getOrCreate()
df = spark.read.option("header",'True').option('delimiter', ',').csv("stocks.csv")
df.printSchema()

24/04/15 09:44:23 WARN Utils: Your hostname, codespaces-7af96c resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/04/15 09:44:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/15 09:44:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/15 09:44:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [4]:
df.show(5)

+----------+----------+----------+------------------+------------------+---------+------------------+
|      Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+----------+----------+----------+------------------+------------------+---------+------------------+
only showing top 5 rows



In [5]:
filtered_df = df.filter((df['Close'] < 500))
selected_df = filtered_df.select('Open', 'Close', 'Close', 'Volume')
selected_df.show()

+------------------+------------------+------------------+---------+
|              Open|             Close|             Close|   Volume|
+------------------+------------------+------------------+---------+
|        213.429998|        214.009998|        214.009998|123432400|
|        214.599998|        214.379993|        214.379993|150476200|
|        214.379993|        210.969995|        210.969995|138040000|
|            211.75|            210.58|            210.58|119282800|
|        210.299994|211.98000499999998|211.98000499999998|111902700|
|212.79999700000002|210.11000299999998|210.11000299999998|115557400|
|209.18999499999998|        207.720001|        207.720001|148614900|
|        207.870005|        210.650002|        210.650002|151473000|
|210.11000299999998|            209.43|            209.43|108223500|
|210.92999500000002|            205.93|            205.93|148516900|
|        208.330002|        215.039995|        215.039995|182501900|
|        214.910006|            21

In [6]:
filtered_df = df.filter((df['Open'] > 200) & (df['Close'] < 200))
filtered_df.show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [7]:

df_with_year = df.withColumn('Year', pyfunc.year(df['Date']), )
df_with_year = df_with_year.select('Year', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close')

df_with_year.show()

+----+----------+------------------+------------------+------------------+------------------+---------+------------------+
|Year|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010|2010-01-11

In [8]:
min_volumes = df_with_year.groupBy('Year').agg(pyfunc.min('Volume').alias('minVolume'))
min_volumes.show()


+----+---------+
|Year|minVolume|
+----+---------+
|2010|100901500|
|2011|100110500|
|2012|100023000|
|2013|100345700|
|2014|100092000|
|2015|101217500|
|2016|110888700|
+----+---------+



In [9]:
df_with_year_month = df_with_year.withColumn('Month', pyfunc.month(df_with_year['Date']))
maxLow = df_with_year_month.groupBy('Year', 'Month').agg(pyfunc.max('Low').alias('maxLow'))
maxLow.show()

+----+-----+------------------+
|Year|Month|            maxLow|
+----+-----+------------------+
|2010|    1|        213.249994|
|2010|    2|        202.000004|
|2010|    3|        234.459999|
|2010|    4|268.19001000000003|
|2010|    5|        262.880009|
|2010|    6|        271.499992|
|2010|    7|        260.300003|
|2010|    8|        260.549995|
|2010|    9|        291.009998|
|2010|   10|        314.289997|
|2010|   11|        316.759987|
|2010|   12|        325.099991|
|2011|    1|        344.440006|
|2011|    2|             360.5|
|2011|    3|        357.750004|
|2011|    4|        350.300007|
|2011|    5|        346.880009|
|2011|    6|        344.649998|
|2011|    7|399.67998900000003|
|2011|    8|        392.369995|
+----+-----+------------------+
only showing top 20 rows



In [10]:
mean_high = df.select(pyfunc.mean('High')).first()[0]
stddev_high = df.select(pyfunc.stddev('High')).first()[0]

print(f"Mean High Price: {mean_high:.2f}")
print(f"Standard Deviation of High Price: {stddev_high:.2f}")

Mean High Price: 315.91
Standard Deviation of High Price: 186.90


In [11]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



24/04/15 09:44:40 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
